## Analysing predictions of Universal Dependencies (POS TAGGING)

http://bionlp-www.utu.fi/dep_search/

In [1]:
import pandas as pd

pd.options.display.max_rows = 100000
pd.set_option('display.max_colwidth', -1)


In [2]:
with open("dev.xlm-roberta-large") as f:
    lines = f.read().split("\n")

all_sentences = []
all_tags = []
sentence = ""
sentene_tags = ""
i = 0
for line in lines:
#     if a sentence ended
    if line=="":
        i += 1
        all_sentences.append(sentence.strip())
        all_tags.append(sentene_tags.strip())
        sentence =""
        sentene_tags = ""
    else:
        word = line.split("\t")[0]
        tag = line.split("\t")[1]
        
        sentence += " "+word
        sentene_tags += " "+tag

In [3]:
zero_shot = True
if zero_shot:
    preds_name = "dev_nl_predictions_zeroshot.txt"
else:
    preds_name = "dev_nl_predictions.txt"

In [4]:
with open(preds_name) as f:
    lines = f.read().split("\n")

all_pred_tags = []
pred_tags = ""
for line in lines:
#     if a sentence ended
    if line=="":
        all_pred_tags.append(pred_tags.strip())
        pred_tags = ""
    else:
        pred_tags += " "+line

In [5]:
for i in range(len(all_tags)):
    actual_len = len(all_tags[i].split()) 
    pred_len = len(all_pred_tags[i].split())
    if not actual_len == pred_len:
        print("index",i)
        merged_pred = all_pred_tags.pop(i)
        all_pred_tags.insert(i," ".join(merged_pred.split()[:actual_len]))
        all_pred_tags.insert(i+1, " ".join(merged_pred.split()[actual_len:]))
#         print("actual tag: ",all_tags[i])
#         print("merged pred tag: ",merged_pred)
#         print("first split tag: "," ".join(merged_pred.split()[:actual_len]))
#         print("second split tag: "," ".join(merged_pred.split()[actual_len:]))

index 772
index 1123
index 1170


In [6]:
ud_pos = pd.DataFrame(data={"sentences":all_sentences,"ground_truth":all_tags,"predictions":all_pred_tags})

In [7]:
unique_tags = "ADJ ADP ADV AUX CCONJ DET INTJ NOUN NUM PART PRON PROPN PUNCT SCONJ SYM VERB X".split()

In [8]:
import numpy as np
confusion_matrix = np.zeros((len(unique_tags),len(unique_tags)))
# rows represent predicted values and columns represent actuals

In [9]:
total_tags = 0
incorrect_tags = 0
mistakes_df = {"predicted":[],"actual":[],"token":[],"sentence":[],"actual_tokens":[]}
for i in range(len(all_tags)):
    actual_sent = all_tags[i].split()
    pred_sent = all_pred_tags[i].split()
    assert len(actual_sent)==len(pred_sent),"Number of actual and predicted tags for the same sentence should be equal"
    for j in range(len(actual_sent)):
        if not actual_sent[j]==pred_sent[j]:
            mistakes_df["predicted"].append(pred_sent[j])
            mistakes_df["actual"].append(actual_sent[j])
            mistakes_df["token"].append(all_sentences[i].split()[j])
            mistakes_df["sentence"].append(all_sentences[i])
            mistakes_df["actual_tokens"].append(" ".join(actual_sent))
            
            confusion_matrix[unique_tags.index(pred_sent[j])][unique_tags.index(actual_sent[j])] += 1
            incorrect_tags += 1
        total_tags += 1
#     if incorrect_tags==5:
#         print("break")
#         break

In [10]:
mistakes_df = pd.DataFrame.from_dict(mistakes_df)

In [11]:
mistakes_df

,predicted,actual,token,sentence,actual_tokens
0,ADV,ADJ,flink,",, Mijn basisniveau is flink omhoog gegaan . ''",PUNCT PRON NOUN AUX ADJ ADV VERB PUNCT PUNCT
1,ADJ,VERB,afgelopen,De afgelopen week twijfelde ze nog of ze wel moest meedoen aan het NK .,DET VERB NOUN VERB PRON ADV SCONJ PRON ADV AUX VERB ADP DET PROPN PUNCT
2,ADV,ADP,op,Moeder Mien spoorde haar aan toch op te stappen .,NOUN PROPN VERB PRON ADP ADV ADP ADP VERB PUNCT
3,PART,ADP,te,Moeder Mien spoorde haar aan toch op te stappen .,NOUN PROPN VERB PRON ADP ADV ADP ADP VERB PUNCT
4,ADV,ADP,halverwege,Tot halverwege de koers had Van Alebeek moeite om zich volledig te concentreren op de wedstrijd .,ADP ADP DET NOUN VERB PROPN PROPN NOUN ADP PRON ADJ ADP VERB ADP DET NOUN PUNCT
5,SCONJ,ADP,om,Tot halverwege de koers had Van Alebeek moeite om zich volledig te concentreren op de wedstrijd .,ADP ADP DET NOUN VERB PROPN PROPN NOUN ADP PRON ADJ ADP VERB ADP DET NOUN PUNCT
6,ADV,ADJ,volledig,Tot halverwege de koers had Van Alebeek moeite om zich volledig te concentreren op de wedstrijd .,ADP ADP DET NOUN VERB PROPN PROPN NOUN ADP PRON ADJ ADP VERB ADP DET NOUN PUNCT
7,PART,ADP,te,Tot halverwege de koers had Van Alebeek moeite om zich volledig te concentreren op de wedstrijd .,ADP ADP DET NOUN VERB PROPN PROPN NOUN ADP PRON ADJ ADP VERB ADP DET NOUN PUNCT
8,ADV,ADP,op,",, Mijn gedachten sprongen alle kanten op .",PUNCT PRON NOUN VERB DET NOUN ADP PUNCT
9,PART,ADP,te,Om ongelukken te voorkomen heb ik mezelf gedwongen om me alleen nog met de koers bezig te houden . '',ADP NOUN ADP VERB AUX PRON PRON VERB ADP PRON ADV ADV ADP DET NOUN ADJ ADP VERB PUNCT PUNCT


In [12]:
from collections import defaultdict
top10_confused_tokens = mistakes_df.token.value_counts().index
for token in top10_confused_tokens:
    print(token)
    token_tags = defaultdict(int)
    display(mistakes_df[mistakes_df.token==token].actual.value_counts())
    for i,sentence in enumerate(all_sentences):
        words = sentence.split()
        for j,word in enumerate(words):
            if word==token:
                tag = all_tags[i].split()[j]
                token_tags[tag] += 1
    print(token_tags)

te


ADP    143
Name: actual, dtype: int64

defaultdict(<class 'int'>, {'ADP': 168, 'ADV': 9})
niet


ADV    79
Name: actual, dtype: int64

defaultdict(<class 'int'>, {'ADV': 101})
er


...

defaultdict(<class 'int'>, {'PROPN': 1})
gekruisigde


VERB    1
Name: actual, dtype: int64

defaultdict(<class 'int'>, {'VERB': 1})
bedroevend


VERB    1
Name: actual, dtype: int64

defaultdict(<class 'int'>, {'VERB': 1})


In [13]:
mistakes_df.token.value_counts()

te                                    143
niet                                  79 
er                                    51 
om                                    43 
zijn                                  32 
en                                    26 
meer                                  25 
dan                                   24 
werd                                  24 
later                                 21 
-                                     20 
als                                   19 
de                                    18 
De                                    17 
was                                   14 
enkele                                13 
heel                                  12 
voor                                  12 
vaak                                  11 
veel                                  11 
zoals                                 11 
is                                    10 
een                                   9  
gisteren                          

In [14]:
conf_df = pd.DataFrame(data=confusion_matrix,index=unique_tags,columns=unique_tags)

#### How many times each tag is misclassified

In [15]:
conf_count = pd.DataFrame()
conf_count["count"] = conf_df.sum().sort_values(ascending=False); 
conf_count["percent"] = conf_count/conf_count.sum()
conf_count[:10]

,count,percent
ADJ,396.0,0.199697
ADP,292.0,0.147252
ADV,273.0,0.137670
NOUN,240.0,0.121029
VERB,226.0,0.113969
PROPN,123.0,0.062027
DET,97.0,0.048916
PRON,90.0,0.045386
SYM,84.0,0.042360
X,62.0,0.031266


In [16]:
sparse_conf_df = conf_df.reset_index().melt(id_vars="index",var_name="actual",value_name="count").rename(columns={"index":"pred"})

sparse_conf_df.sort_values(by="count",ascending=False)

,pred,actual,count
2,ADV,ADJ,270.0
130,PROPN,NOUN,191.0
26,PART,ADP,143.0
44,PRON,ADV,100.0
255,ADJ,VERB,96.0
43,PART,ADV,81.0
258,AUX,VERB,80.0
19,ADV,ADP,75.0
11,PROPN,ADJ,66.0
30,SCONJ,ADP,61.0


In [17]:
total_tags, incorrect_tags

(22938, 1983)

In [18]:
print("Accuracy: ",(total_tags-incorrect_tags)/total_tags)

Accuracy:  0.9135495684017787


In [19]:
ud_pos.tail()

,sentences,ground_truth,predictions
1392,Vandersteen wekte de sympathie op van zijn collega's en afdelingschef en mocht het vakblad van de slagers illustreren .,PROPN VERB DET NOUN ADP ADP PRON NOUN CCONJ NOUN CCONJ AUX DET NOUN ADP DET NOUN VERB PUNCT,PROPN VERB DET NOUN ADP ADP PRON NOUN CCONJ NOUN CCONJ AUX DET NOUN ADP DET NOUN VERB PUNCT
1393,Ook de wachtkamer en de personeels- en vergaderzalen mocht hij onder handen nemen .,ADV DET NOUN CCONJ DET X CCONJ NOUN AUX PRON ADP NOUN VERB PUNCT,ADV DET NOUN CCONJ DET NOUN CCONJ NOUN AUX PRON ADP NOUN VERB PUNCT
1394,"Zo kwam hij in contact met J. Meuwissen , een Nederlander die sinds 1936 het blad Bravo uitgaf en op zoek was naar Vlaams en Nederlands tekentalent nu er geen Amerikaanse strips meer mochten gepubliceerd worden .",ADV VERB PRON ADP NOUN ADP PROPN PROPN PUNCT DET PROPN PRON ADP NUM DET NOUN PROPN VERB CCONJ ADP NOUN AUX ADP ADJ CCONJ ADJ NOUN SCONJ ADV DET ADJ NOUN ADV AUX VERB AUX PUNCT,ADV VERB PRON ADP NOUN ADP PROPN PROPN PUNCT DET NOUN PRON ADP NUM DET NOUN PROPN VERB CCONJ ADP NOUN AUX ADP ADJ CCONJ ADJ NOUN SCONJ PRON DET ADJ NOUN ADV AUX VERB AUX PUNCT
1395,"Vandersteen mocht nu wekelijks een pagina vol tekenen rond Â« Sindbad de zeerover Â» , Â« Thor de holbewoner Â» en Â« Lancelot Â» .",PROPN AUX ADV ADJ DET NOUN ADJ VERB ADP PUNCT PROPN DET NOUN PUNCT PUNCT PUNCT PROPN DET NOUN PUNCT CCONJ PUNCT PROPN PUNCT PUNCT,PROPN AUX ADV ADV DET NOUN ADP NOUN ADP PUNCT PROPN DET PROPN PUNCT PUNCT PUNCT PROPN DET NOUN PUNCT CCONJ PUNCT PROPN PUNCT PUNCT
1396,"Het waren stuk voor stuk erg onbeholpen , maar charmante strips die de lezers wel op de lachspieren werkten .",PRON AUX NOUN ADP NOUN ADJ ADJ PUNCT CCONJ ADJ NOUN PRON DET NOUN ADV ADP DET NOUN VERB PUNCT,PRON AUX NOUN ADP NOUN ADV ADJ PUNCT CCONJ ADJ NOUN PRON DET NOUN ADV ADP DET NOUN VERB PUNCT
